# Introductory Notebook 2: Mixed-Datatype Problems, Multi-Objective and Mixed-Objective Problems
This notebook will cover mixed datatypes and generating counterfactuals with multiple mixed objectives types.

In [16]:
import sys
sys.path.append('../src/')
import decode_mcd
from decode_mcd import mcd_problem
from decode_mcd import mcd_dataset


from decode_mcd import design_targets
from decode_mcd import mcd_generator

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Creating a Dataset

First, lets create a dataset inspired by some basic arithmetic. This time, we will create a dataset with four variables of different types. `A` will be a random integer from 0 to 10 inclusive. `B` will be a random float from -1 to 1. `C` will be a random choice among "Add" "Subtract", "Multiply", and "Divide". Finally, `D` will be a boolean variable determining the sign of the expression. True will correspond to positive while False will mean negative. To effectively handle mixed datatypes will will use pandas.

In [17]:
num_data = 1000
A = np.random.randint(0, 10, num_data)
B = np.random.rand(num_data) * 2 - 1 #Randomized values which originally range from 0 to 1. Scales to -1 to 1.
C = np.random.choice(["Add", "Subtract", "Multiply", "Divide"], num_data)
D = np.random.choice([True, False], num_data)
x = pd.DataFrame({"A": A, "B": B, "C": C, "D": D})
display(x)


,A,B,C,D
0,0,-0.146610,Add,False
1,6,-0.518038,Add,False
2,5,-0.286447,Divide,True
3,4,0.575552,Subtract,False
4,3,-0.118617,Divide,True
...,...,...,...,...
995,4,-0.301676,Divide,True
996,3,0.796996,Add,False
997,5,0.215525,Divide,True
998,9,0.510222,Divide,True


We will create two functions. In the first one we will perform operation C(D(A), B). The second will perform D(C(B, A))>=0. For example, if C is add and D is False, function 1 will calculate -A+B while function 2 will check if -(B+A) is greater than 0 or not. We code up the `evaluate` function which takes in a dataframe with a set of `A`, `B`, `C`, and `D` values and returns a set of `O1` and `O2` values, which is what we call our function

In [18]:
def apply_operation(C, x, y):
    #Vectorized function to calculate C(x, y), for example if C is "Subtract", calculates x-y.
    add_mask = (C == "Add")
    subtract_mask = (C == "Subtract")
    multiply_mask = (C == "Multiply")
    divide_mask = (C == "Divide")
    result = np.zeros(len(C))
    result[add_mask] = x[add_mask] + y[add_mask]
    result[subtract_mask] = x[subtract_mask] - y[subtract_mask]
    result[multiply_mask] = x[multiply_mask] * y[multiply_mask]
    result[divide_mask] = x[divide_mask] / y[divide_mask]
    return result

def apply_inverse(D, x):
    #Vectorized function to calculate D(x), for example if D is False, returns -x. 
    return x*D-x*~D

def evaluate(x):
    #Evaluation function to calculate both objectives. x is an nx4 dataframe. 
    A = x["A"] #First isolate the individual variables from the provided dataframe
    B = x["B"]
    C = x["C"]
    D = x["D"].astype(bool)
    objective_1 = apply_operation(C, apply_inverse(D, A), B) #Calculate objective 1
    objective_2 = np.greater(apply_inverse(D, apply_operation(C, B, A)), 0) #Calculate objective 2
    return pd.DataFrame({"O1": objective_1, "O2": objective_2}) #Create a nx2 dataframe with the objective values
 

Let's evaluate our dataset.

In [19]:
y = evaluate(x)
display(y)

,O1,O2
0,-0.146610,True
1,-6.518038,False
2,-17.455246,False
3,-4.575552,True
4,-25.291478,False
...,...,...
995,-13.259270,False
996,-2.203004,False
997,23.199174,True
998,17.639393,True


Finally, we create our query. 

In [20]:
x_query = pd.DataFrame({"A": [0], "B": [0.0], "C": ["Add"], "D": [True]}, index = ["Query"])
display(x_query)

,A,B,C,D
Query,0,0.0,Add,True


### Setting up MCD

Now we are ready to set up the `McdDataset`. We specify the datatypes of our dataset. This time we have one of each type of variable. We specify the boundaries and options as specified earlier in the notebook.

In [21]:
from pymoo.core.variable import Real, Integer, Choice, Binary 
datatypes=[Integer(bounds=(0, 10)), 
           Real(bounds=(-1, 1)), 
           Choice(options=["Add", "Subtract", "Multiply", "Divide"]), 
           Binary()]

data = mcd_dataset.McdDataset(x=x, y=y, x_datatypes=datatypes)

Next, we create the design targets and the `McdProblem`. In this case, we create two design targets. We have one continuous target and one categorical target. Continuous targets should be used for any objective with ordinal significance (such as floats or ints), while categorical targets should be used for those without (such as classes or bools). We want set a hard lower bound of 10 for O1 with no upper bound, meaning C(D(A), B)>=10. We also set a requirement that O2 must be true, meaning that D(C(B, A))>=0

In [22]:
#We first set up a ContinuousTarget for O1, setting a minimum of 10 and setting no upper bound (i.e. infinity))
target_1 = design_targets.ContinuousTarget(label = "O1", lower_bound=10, upper_bound=np.inf)

#We then set up a CategoricalTarget for O2 specifying only True as the desired class. 
#Desired_classes is a list. In problems with multiple classes, this list specifies the acceptable classes.
target_2 = design_targets.CategoricalTarget(label = "O2", desired_classes=[True])

#We then create a DesignTargets object with the two targets.
y_targets = design_targets.DesignTargets(continuous_targets=[target_1], categorical_targets=[target_2])

In [23]:
problem = mcd_problem.McdProblem(mcd_dataset=data, x_query = x_query, y_targets = y_targets, prediction_function=evaluate)

Finally, we create the `McdGenerator`:

In [24]:
generator = mcd_generator.McdGenerator(problem=problem, pop_size=100, initialize_from_dataset=True)

### Generating Counterfactuals
Finally, we run the generator and sample.

In [25]:
generator.generate(n_generations=10)

1000 dataset entries found matching problem parameters
Initial population initialized from dataset of 1000 samples!
Training GA from 0 to 10 generations!
n_gen  |  n_eval  | n_nds  |     cv_min    |     cv_avg    |      eps      |   indicator  
     1 |        0 |      3 |  0.000000E+00 |  1.288679E+01 |             - |             -
     2 |      100 |      3 |  0.000000E+00 |  0.2792456453 |  0.000000E+00 |             f
     3 |      200 |      4 |  0.000000E+00 |  0.0112724012 |  0.0171422270 |             f


c:\Users\Lyler\Documents\Multiobjective-Counterfactuals-for-Design\Getting Started\../src\decode_mcd\mcd_problem.py:301: RuntimeWarning: invalid value encountered in subtract
  satisfaction = np.maximum(actual - query_ub, query_lb - actual)


     4 |      300 |      4 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
     5 |      400 |      5 |  0.000000E+00 |  0.000000E+00 |  1.0000000000 |         ideal
     6 |      500 |      5 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
     7 |      600 |      5 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f


c:\Users\Lyler\Documents\Multiobjective-Counterfactuals-for-Design\Getting Started\../src\decode_mcd\mcd_problem.py:301: RuntimeWarning: invalid value encountered in subtract
  satisfaction = np.maximum(actual - query_ub, query_lb - actual)
c:\Users\Lyler\Documents\Multiobjective-Counterfactuals-for-Design\Getting Started\../src\decode_mcd\mcd_problem.py:301: RuntimeWarning: invalid value encountered in subtract
  satisfaction = np.maximum(actual - query_ub, query_lb - actual)


     8 |      700 |      5 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
     9 |      800 |      5 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
    10 |      900 |      5 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f


c:\Users\Lyler\Documents\Multiobjective-Counterfactuals-for-Design\Getting Started\../src\decode_mcd\mcd_problem.py:301: RuntimeWarning: invalid value encountered in subtract
  satisfaction = np.maximum(actual - query_ub, query_lb - actual)


In [26]:
num_samples = 10 
counterfactuals = generator.sample(num_samples, include_dataset=False)
display(counterfactuals)

Scoring all counterfactual candidates!
Calculating diversity matrix!
Sampling diverse set of counterfactual candidates!
samples_index=[np.int64(57), np.int64(153), np.int64(170), np.int64(188), np.int64(46), np.int64(78), np.int64(226), np.int64(187), np.int64(129), np.int64(39)]
Done! Returning CFs


,A,B,C,D
0,1,0.069608,Divide,True
1,10,0.000000,Add,True
2,5,0.004808,Divide,True
3,10,0.050370,Add,True
4,2,0.182860,Divide,True
5,3,0.081853,Divide,True
6,2,0.012929,Divide,True
7,4,0.377121,Divide,True
8,8,0.189402,Divide,True
9,3,0.269602,Divide,True


Let's evaluate the counterfactuals we generated, we should see that every O1 value is greater or equal to 10, while every O2 value is true. 

In [27]:
evaluate(counterfactuals)

,O1,O2
0,14.366146,True
1,10.000000,True
2,1039.920161,True
3,10.050370,True
4,10.937324,True
5,36.650861,True
6,154.689411,True
7,10.606663,True
8,42.238248,True
9,11.127507,True


### Adding Optimization Objectives to the Mix
Now let's try converting one of our two constraints into an objective. We will make our `ContinuousTarget` into a `MinimizationTarget`.  In this case, we do not specify bounds, but if we wanted to further contstraint the objective to minimize, this could be done with a `ContinuousTarget` and a `MinimizationTarget`. 

In [28]:
target_1 = design_targets.MinimizationTarget(label = "O1")
y_targets = design_targets.DesignTargets(minimization_targets=[target_1], categorical_targets=[target_2])
problem = mcd_problem.McdProblem(mcd_dataset=data, x_query = x_query, y_targets = y_targets, prediction_function=evaluate)

In [29]:
generator = mcd_generator.McdGenerator(problem=problem, pop_size=100, initialize_from_dataset=True)
generator.generate(n_generations=10)
counterfactuals = generator.sample(num_samples, include_dataset=False)

1000 dataset entries found matching problem parameters
Initial population initialized from dataset of 1000 samples!
Training GA from 0 to 10 generations!
n_gen  |  n_eval  | n_nds  |     cv_min    |     cv_avg    |      eps      |   indicator  
     1 |        0 |     67 |  0.000000E+00 |  0.5644355644 |             - |             -
     2 |      100 |     78 |  0.000000E+00 |  0.000000E+00 |  0.0368670858 |         ideal
     3 |      200 |     95 |  0.000000E+00 |  0.000000E+00 |  0.3093270217 |         nadir
     4 |      300 |    100 |  0.000000E+00 |  0.000000E+00 |  0.0165442754 |             f
     5 |      400 |    100 |  0.000000E+00 |  0.000000E+00 |  0.0033363981 |             f
     6 |      500 |    100 |  0.000000E+00 |  0.000000E+00 |  0.0055261594 |             f
     7 |      600 |    100 |  0.000000E+00 |  0.000000E+00 |  0.0083994983 |             f
     8 |      700 |    100 |  0.000000E+00 |  0.000000E+00 |  0.0066662324 |         ideal
     9 |      800 |    100 

We can see that in this case, the generated counterfactuals have much smaller O1 values. 

In [30]:
evaluate(counterfactuals)

,O1,O2
0,9.190999,True
1,0.997043,True
2,-4.915329,True
3,0.252113,True
4,0.554939,True
5,7.951171,True
6,6.021966,True
7,-0.007731,True
8,-0.018382,True
9,0.958834,True


This concludes the second MCD tutorial notebook! The third will cover advanced selection options for counterfactuals. 